In [1]:
import xml.etree.ElementTree as ET
from xml.dom import minidom
%config Completer.use_jedi = False
import utils.db as db
import glob
import datetime
import folia.main as folia
from utils import db
mongo = db.get_db()


1. Download a fragment
2. Check if USC
3. If so get all transcripts
4. Get a word sequence from it
5. Find it in the transcripts

In [7]:
def get_all_fragments():
    mongo = db.get_db()
    results = mongo.fragments.find({},{'_id':0})
    results = [element for element in results]
    all_mid_nodes =[]
    all_fragments = []
    for result in results:
        mid_nodes = [element for element in result['tree']['children'] if len(element['children'])>0]
        for mid_node in mid_nodes:

            all_mid_nodes.append(mid_node['label'])
            for fragment in mid_node['children']:
                all_fragments.append({'fragment':fragment,'mid_label':mid_node['label'],'top_label':result['tree']['label']})
            
    return (all_fragments)

In [103]:
def update_fragments(main_node_label,mid_node_label,fragment,attributes):
    results = mongo.fragments.find({'label':main_node_label},{'_id':0})[0]
    mongo_id= mongo.fragments.find({'label':main_node_label},{'_id':1})[0]
    for x,element in enumerate(results['tree']['children']):
        if element['label']==mid_node_label:
            for n,leaves in enumerate(element['children']):
                if leaves['label'] == fragment['label']:
                    for attr in attributes:
                        results['tree']['children'][x]['children'][n][list(attr.keys())[0]]=attr[list(attr.keys())[0]]
                    
    replace = mongo.fragments.replace_one(mongo_id,results)


In [102]:
update_fragments("rape","kill",fragment['fragment'],[{'media_index':100}])

{'label': 'Sometimes, they raped them. They killed them right away.', 'testimony_id': 'usc_shoah_2590', 'media_index': 100, 'media_offset': 0.0, 'start_sentence_index': 132.0, 'end_sentence_index': 134.0, 'children': []}


In [8]:
all_fragments = get_all_fragments()

In [9]:
for fragment in all_fragments:
    if 'usc' in fragment['fragment']['testimony_id'].split('_'):
        testimony_id = fragment['fragment']['testimony_id']
        testimony_id  ='usc_shoah_10'
        print (fragment)
        sentence_start = fragment['fragment']['start_sentence_index']
        p1 = minidom.parse("Data/folia_output/"+testimony_id+'.xml')
        sentences = p1.getElementsByTagName('s')
        src_attr = sentences[int(sentence_start)].parentNode.getAttribute('src')
        part= src_attr.split('_')[0]
        time = src_attr.split('_')[1]
        
        break
        
        

        

{'fragment': {'label': 'Sometimes, they raped them. They killed them right away.', 'testimony_id': 'usc_shoah_2590', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 132.0, 'end_sentence_index': 134.0, 'children': []}, 'mid_label': 'kill', 'top_label': 'rape'}


In [38]:
sentences[int(sentence_start)].parentNode.getAttribute('src')

'1_1215'

In [278]:
unresolved = []
for fragment in all_fragments:
    if 'usc' in fragment['fragment']['testimony_id'].split('_'):
        testimony_id = fragment['fragment']['testimony_id'].split('_')[-1]
        print (fragment['fragment'])
        if (fragment['fragment']['label'].strip()[0] == "("):
            fragment_parts = fragment['fragment']['label'].strip()[4:].strip().split(' ')[0:5]
            print (fragment_parts)
            
        elif (fragment['fragment']['label'].strip()[0] == "."):
            fragment_parts = fragment['fragment']['label'].strip()[3:].strip().split(' ')[0:5]
            print (fragment_parts)
            continue
        fragment_parts = fragment['fragment']['label'].strip().split(' ')[0:5]
        #print(testimony_id)
        files = glob.glob("data/transcripts/"+testimony_id+".*.xml")
        final_result = []
        for file in files:
            result = check_fragment_location(file,fragment_parts)
            final_result.append(result)
            if result !=False:
                #print (testimony_id)
                #print (result)
                #print (file)
                part = file.split('.')[-2]
                #print (part)
                #print (result)
                #print (fragment['fragment']['label'])
        if len([element for element in final_result if element!=False])!=1:
            #print (final_result)
            #print (testimony_id)
            #print (fragment['fragment']['label'])
            unresolved.append({'id':testimony_id,'label':fragment['fragment']['label']})
        
        

{'label': 'Sometimes, they raped them. They killed them right away.', 'testimony_id': 'usc_shoah_2590', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 132.0, 'end_sentence_index': 134.0, 'children': []}
{'label': 'they burned, they killed, they raped,', 'testimony_id': 'usc_shoah_7458', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 436.0, 'end_sentence_index': 437.0, 'children': []}
{'label': 'She saw her mother raped to death', 'testimony_id': 'usc_shoah_14900', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 1208.0, 'end_sentence_index': 1208.0, 'children': []}
{'label': ' And I watched them rape the women.', 'testimony_id': 'usc_shoah_162', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 599.0, 'end_sentence_index': 600.0, 'children': []}
{'label': " She saw, she-- that's the one, she was raped", 'testimony_id': 'usc_shoah_1222', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 197.0, 'end_sentenc

{'label': "There were, I would say, 70 to 80 people to-- to a-- a-- a cattle truck. Some couldn't sit down, others could.", 'testimony_id': 'usc_shoah_4187', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 692.0, 'end_sentence_index': 693.0, 'children': []}
{'label': "Anyhow, the following day, they load us up on trucks. And we were going, we don't know where. ", 'testimony_id': 'usc_shoah_20084', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 655.0, 'end_sentence_index': 657.0, 'children': []}
{'label': 'You know, if it was a week or two weeks, to sit in the cattle trucks.', 'testimony_id': 'usc_shoah_13524', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 2023.0, 'end_sentence_index': 2024.0, 'children': []}
{'label': 'It took me many years to understand that I was sitting surrounded by dead bodies. I was sitting on dead bodies.', 'testimony_id': 'usc_shoah_19518', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 155

{'label': 'I was all numb. I lost everybody.', 'testimony_id': 'usc_shoah_198', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 733.0, 'end_sentence_index': 736.0, 'children': []}
{'label': "I couldn't even cry anymore. Just couldn't. I mean, there just-- I was numb.", 'testimony_id': 'usc_shoah_22892', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 701.0, 'end_sentence_index': 704.0, 'children': []}
{'label': "I probably was numb; I couldn't cry", 'testimony_id': 'usc_shoah_6845', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 462.0, 'end_sentence_index': 462.0, 'children': []}
{'label': "We did not recognize each other. You know, we're just standing there completely naked with bald head.", 'testimony_id': 'usc_shoah_10162', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 887.0, 'end_sentence_index': 889.0, 'children': []}
{'label': 'And we said this is an insane asylum. And we hardly recognized each other.', 'testi

{'label': 'Hit harder and harder.\xa0The old man died.', 'testimony_id': 'usc_shoah_964', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 816.0, 'end_sentence_index': 818.0, 'children': []}
{'label': 'Dr. Elkis was almost, like, 70 years old.\xa0He slapped him in the face.\xa0', 'testimony_id': 'usc_shoah_18626', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 3457.0, 'end_sentence_index': 3459.0, 'children': []}
{'label': '...this old man was beaten up in the ghetto before he was thrown into the-- to the train...', 'testimony_id': 'usc_shoah_8786', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 46.0, 'end_sentence_index': 46.0, 'children': []}
['this', 'old', 'man', 'was', 'beaten']
{'label': '...they beat my father with this. ', 'testimony_id': 'usc_shoah_10012', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 558.0, 'end_sentence_index': 559.0, 'children': []}
['they', 'beat', 'my', 'father', 'with']
{'label': 'I h

{'label': 'And I was under that bed, lying flat, afraid to breathe even.', 'testimony_id': 'usc_shoah_9590', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 629.0, 'end_sentence_index': 630.0, 'children': []}
{'label': 'Because of fear, we were absolutely silent, almost afraid to breathe, ', 'testimony_id': 'usc_shoah_8784', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 430.0, 'end_sentence_index': 430.0, 'children': []}
{'label': "And she couldn't take it out, and the baby couldn't breathe, and the baby choked.", 'testimony_id': 'usc_shoah_25381', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 1249.0, 'end_sentence_index': 1250.0, 'children': []}
{'label': "And she choked the baby. Got choked. Couldn't breathe because they were a long time in the forest.", 'testimony_id': 'usc_shoah_25381', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 1245.0, 'end_sentence_index': 1248.0, 'children': []}
{'label': 'we were eatin

...

{'label': 'So she grabbed her and embraced her. And she started to cry (..)', 'testimony_id': 'usc_shoah_14190', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 1085.0, 'end_sentence_index': 1086.0, 'children': []}
{'label': 'He embraces me, and we both cry.', 'testimony_id': 'usc_shoah_268', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 3266.0, 'end_sentence_index': 3267.0, 'children': []}
{'label': 'Yeah, well I smelled dead bodies and stench and I mean, the place smelled.', 'testimony_id': 'usc_shoah_13483', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 1276.0, 'end_sentence_index': 1277.0, 'children': []}
{'label': 'It was a terrible smell, hair and bodies. You could smell. And was very scary.', 'testimony_id': 'usc_shoah_15610', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 792.0, 'end_sentence_index': 795.0, 'children': []}
{'label': 'These thousands of dead bodies piled up and the smell, the stink (..).', 

{'label': ' And they used to have to count them 25. They used to get 25 whips.', 'testimony_id': 'usc_shoah_1496', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 336.0, 'end_sentence_index': 338.0, 'children': []}
{'label': 'They used to get 25 whips. And if they missed it, they had to count all over again. The whips would cut the flesh.', 'testimony_id': 'usc_shoah_1496', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 337.0, 'end_sentence_index': 340.0, 'children': []}
{'label': 'You were not allowed to pick up your head from the work that did. If you did, you had a whip around your neck.', 'testimony_id': 'usc_shoah_27443', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 705.0, 'end_sentence_index': 707.0, 'children': []}
{'label': 'But they were like-- they used like those whips-- work, work, work, work.', 'testimony_id': 'usc_shoah_3474', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 348.0, 'end_sentence_index'

{'label': 'And the Jews are not allowed to walk on the street until at least they-- if they have jobs.', 'testimony_id': 'usc_shoah_10960', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 332.0, 'end_sentence_index': 333.0, 'children': []}
{'label': "They wouldn't allow any Jews walking at all in the streets.", 'testimony_id': 'usc_shoah_19111', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 626.0, 'end_sentence_index': 627.0, 'children': []}
{'label': ' And he walked and walked and walked, for days and days and days. He walked nights (..).', 'testimony_id': 'usc_shoah_1765', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 359.0, 'end_sentence_index': 360.0, 'children': []}
{'label': 'And we walked, and we walked for days and nights.', 'testimony_id': 'usc_shoah_4967', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 1604.0, 'end_sentence_index': 1605.0, 'children': []}
{'label': 'They had not the strength to walk out.

{'label': 'And they put us naked if -- in, in a like a big stable was there. And we were there a whole night...', 'testimony_id': 'usc_shoah_24130', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 2243.0, 'end_sentence_index': 2244.0, 'children': []}
{'label': 'And there we were standing, five of us, nude on the middle of that place. And he laughing. ', 'testimony_id': 'usc_shoah_27687', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 1278.0, 'end_sentence_index': 1280.0, 'children': []}
{'label': " ...there 's snow. And the next building is 50 feet away. And I 'm -- I 'm stark naked. ", 'testimony_id': 'usc_shoah_10549', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 560.0, 'end_sentence_index': 563.0, 'children': []}
['there', "'s", 'snow.', 'And', 'the']
{'label': '...I remember we had to stand naked there. We had to crawl in the snow ...', 'testimony_id': 'usc_shoah_11000', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_

{'label': '...the other guy had the stamina to run to the wires and take his life...', 'testimony_id': 'usc_shoah_22869', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 815.0, 'end_sentence_index': 815.0, 'children': []}
['the', 'other', 'guy', 'had', 'the']
{'label': 'And then I ran into the forest among the trees. ', 'testimony_id': 'usc_shoah_10358', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 730.0, 'end_sentence_index': 731.0, 'children': []}
{'label': 'We were running, running for our life all the time.', 'testimony_id': 'usc_shoah_1530', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 1221.0, 'end_sentence_index': 1222.0, 'children': []}
{'label': 'Take jewelry.\xa0Take food.\xa0But nobody thought about it, because everybody was running for-- for their lives.\xa0', 'testimony_id': 'usc_shoah_16887', 'media_index': 0.0, 'media_offset': 0.0, 'start_sentence_index': 271.0, 'end_sentence_index': 274.0, 'children': []}
{'label':

In [277]:
len(unresolved)

337

In [236]:
for i,element in enumerate(result):
    if element.firstChild.nodeValue =="There":
        print(result[i+2].firstChild.nodeValue)
    

a
one
no
no
no
still
nothing
Germany.


In [214]:
files

['data/transcripts/11184.1.xml',
 'data/transcripts/11184.2.xml',
 'data/transcripts/11184.3.xml',
 'data/transcripts/11184.4.xml']

In [212]:
unresolved

[{'id': '11184', 'label': 'There was no sleep, not much sleep at all.'}]

In [167]:
for i,element in enumerate(spans):
        try:
            first = element.firstChild.nodeValue
            second = spans[i+1].firstChild.nodeValue
            third = spans[i+2].firstChild.nodeValue
        except Exception as e:
            continue
        if (first ==fragment_parts[0]):
            print (third)
        if (first ==fragment_parts[0]) and (second==fragment_parts[1]) and (third ==fragment_parts[2]):
            millis = (element.getAttribute('m'))
            d = datetime.timedelta(milliseconds=int(millis))
            result =  d.seconds
            break
        else:
            result = False
    

In [203]:
unresolved

[{'id': '1222', 'label': " She saw, she-- that's the one, she was raped"},
 {'id': '11593',
  'label': '(..) name was Hannah Prusak, P-R-U-S-A-K-- was raped and shot in the middle of the street.'},
 {'id': '11184', 'label': 'There was no sleep, not much sleep at all.'},
 {'id': '7455', 'label': 'I was afraid to sleep, if my dreams control me.'},
 {'id': '5103', 'label': 'And I felt like my whole world had collapsed (..).'},
 {'id': '7093',
  'label': 'anybody who collapsed (..) they were shot on the spot and left there.'},
 {'id': '7093',
  'label': 'anybody who collapsed (..) they were shot on the spot and left there. '},
 {'id': '10710',
  'label': 'One day, I was sitting after work in the bunk there, in the barrack, and I collapsed.'},
 {'id': '3341',
  'label': '(..) I was so sick that I collapsed in the working trenches, where we were working.'},
 {'id': '11272',
  'label': "And then the next thing, the elderly couldn't stand on their feet. They started collapsing."},
 {'id': '194

In [148]:
unresolved[]

[{'id': '162', 'label': ' And I watched them rape the women.'},
 {'id': '1222', 'label': " She saw, she-- that's the one, she was raped"},
 {'id': '10406',
  'label': 'They probably raped them, abused them, eventually shot them because nobody survived'},
 {'id': '11593',
  'label': '(..) name was Hannah Prusak, P-R-U-S-A-K-- was raped and shot in the middle of the street.'},
 {'id': '11184', 'label': 'There was no sleep, not much sleep at all.'},
 {'id': '7455', 'label': 'I was afraid to sleep, if my dreams control me.'},
 {'id': '151', 'label': ' Our world collapsed. Our life has changed forever.'},
 {'id': '5103', 'label': 'And I felt like my whole world had collapsed (..).'},
 {'id': '10710',
  'label': 'One day, I was sitting after work in the bunk there, in the barrack, and I collapsed.'},
 {'id': '3341',
  'label': '(..) I was so sick that I collapsed in the working trenches, where we were working.'},
 {'id': '11272',
  'label': "And then the next thing, the elderly couldn't stan

In [114]:
files

['data/transcripts/2590.1.xml',
 'data/transcripts/2590.2.xml',
 'data/transcripts/2590.3.xml']

In [84]:
spans = result
for i,element in enumerate(result):
        try:
            first = element.firstChild.nodeValue
            second = spans[i+1].firstChild.nodeValue
            third = spans[i+2].firstChild.nodeValue
        except Exception as e:
            print (e)
            continue
        if (first ==fragment_parts[0]) and (second==fragment_parts[1]) and (third ==fragment_parts[2]):
            millis = (element.getAttribute('m'))
            d = datetime.timedelta(milliseconds=int(millis))
            print (d.seconds)

581
list index out of range
list index out of range


In [72]:
result

[<DOM Element: span at 0x10f0fb9c8>,
 <DOM Element: span at 0x10f0fba60>,
 <DOM Element: span at 0x10f0fbaf8>,
 <DOM Element: span at 0x10f0fbb90>,
 <DOM Element: span at 0x10f0fbc28>,
 <DOM Element: span at 0x10f0fbcc0>,
 <DOM Element: span at 0x10f0fbd58>,
 <DOM Element: span at 0x10f0fbdf0>,
 <DOM Element: span at 0x10f0fbe88>,
 <DOM Element: span at 0x10f0fbf20>,
 <DOM Element: span at 0x10f105048>,
 <DOM Element: span at 0x10f1050e0>,
 <DOM Element: span at 0x10f105178>,
 <DOM Element: span at 0x10f105210>,
 <DOM Element: span at 0x10f1052a8>,
 <DOM Element: span at 0x10f105340>,
 <DOM Element: span at 0x10f1053d8>,
 <DOM Element: span at 0x10f105470>,
 <DOM Element: span at 0x10f105508>,
 <DOM Element: span at 0x10f1055a0>,
 <DOM Element: span at 0x10f105638>,
 <DOM Element: span at 0x10f1056d0>,
 <DOM Element: span at 0x10f105768>,
 <DOM Element: span at 0x10f105800>,
 <DOM Element: span at 0x10f105898>,
 <DOM Element: span at 0x10f105930>,
 <DOM Element: span at 0x10f1059c8>,
 

In [53]:
files

['data/transcripts/2590.1.xml',
 'data/transcripts/2590.2.xml',
 'data/transcripts/2590.3.xml']

In [54]:
fragment

{'fragment': {'label': 'Sometimes, they raped them. They killed them right away.',
  'testimony_id': 'usc_shoah_2590',
  'media_index': 0.0,
  'media_offset': 0.0,
  'start_sentence_index': 132.0,
  'end_sentence_index': 134.0,
  'children': []},
 'mid_label': 'kill',
 'top_label': 'rape'}

In [33]:
files

['data/transcripts/2590.1.xml',
 'data/transcripts/2590.2.xml',
 'data/transcripts/2590.3.xml']

In [ ]:
fragment = "But again, SS shot them. And when people collapsed, they were shot. There was will in me."
words_to_identify = fragment.split(' ')[0:3]
testimony_id = 'usc_shoah_5758'.split('_')[-1]

In [ ]:
testimony_id

In [ ]:
p1 = minidom.parse("data/transcripts/"+testimony_id+".5.xml")

In [ ]:
spans = p1.getElementsByTagName('span')

In [ ]:
len(spans)

In [ ]:
import datetime
for i,element in enumerate(spans):
    try:
        first = element.firstChild.nodeValue
        second = spans[i+1].firstChild.nodeValue
        third = spans[i+2].firstChild.nodeValue
    except:
        continue
    if (first =="But") and (second=="again,") and (third =="SS"):
        millis = (element.getAttribute('m'))
        d = datetime.timedelta(milliseconds=int(millis))
        print (d.seconds)

        

    

In [ ]:
for x in myroot.findall('p'):
    print (x.text)